#### Import

Import the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import requests
api='[YOUR API KEY]'

from datetime import datetime
from statistics import mean

#### Load the dataset
Read the data from the given file path. 

In [ ]:
df=pd.read_csv ('/content/preprocessed.csv')

#### List the addresses to complement
Create a list of addresses from the dataframe `df` where the `Total_ERC20_tnxs` column is null.

In [ ]:
address_to_complement= df[df['Total_ERC20_tnxs'].isnull()].Address.to_list()

#### Iterate through the list and make API call
Iterate through the list of addresses and make an API call to the Etherscan API for each address. If the address did not have a transaction we complete the list with `0` 

In [ ]:
address_reviewed=[]
Total_ERC20_tnxs=[]
ERC20_total_Ether_received=[]
ERC20_total_ether_sent=[]
ERC20_total_Ether_sent_contract=[]
ERC20_uniq_sent_addr=[]
ERC20_uniq_rec_addr=[]
ERC20_uniq_rec_contract_addr=[]
ERC20_min_val_rec=[]
ERC20_avg_val_rec=[]
ERC20_uniq_sent_token_name=[]
new_list_to_iterate=[]

for add in address_to_complement:
  time.sleep (1)
  url= 'https://api.etherscan.io/api?module=account&action=tokentx&address='+add+'&startblock=0&endblock=17059560&sort=asc&apikey='+api
  response= requests.get (url)
  if response.json()['message']=='No transactions found':
    address_reviewed. append (add)
    Total_ERC20_tnxs.append (0)
    ERC20_total_Ether_received.append (0)
    ERC20_total_ether_sent.append (0)
    ERC20_total_Ether_sent_contract.append (0)
    ERC20_uniq_sent_addr.append (0)
    ERC20_uniq_rec_addr.append (0)
    ERC20_uniq_rec_contract_addr.append (0)
    ERC20_min_val_rec.append (0)
    ERC20_avg_val_rec.append (0)
    ERC20_uniq_sent_token_name.append (0)
  else:
    new_list_to_iterate.append (add)

#### Creating a DataFrame
Create a dataframe from the variables listed above. 

In [ ]:
df_iterated = pd.DataFrame(list(zip(address_reviewed, Total_ERC20_tnxs, ERC20_total_Ether_received, ERC20_total_ether_sent, ERC20_total_Ether_sent_contract, ERC20_uniq_sent_addr, ERC20_uniq_rec_addr,
                           ERC20_uniq_rec_contract_addr, ERC20_min_val_rec, ERC20_avg_val_rec, ERC20_uniq_sent_token_name)),
               columns =['address_reviewed', 'Total_ERC20_tnxs', 'ERC20_total_Ether_received', 'ERC20_total_ether_sent', 'ERC20_total_Ether_sent_contract', 'ERC20_uniq_sent_addr', 'ERC20_uniq_rec_addr',
                           'ERC20_uniq_rec_contract_addr', 'ERC20_min_val_rec', 'ERC20_avg_val_rec', 'ERC20_uniq_sent_token_name'])

df_iterated.head()
#df_iterated.to_csv ('iteratedcolumns.csv', index=False)

,address_reviewed,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_rec_contract_addr,ERC20_min_val_rec,ERC20_avg_val_rec,ERC20_uniq_sent_token_name
0,0x0020731604c882cf7bf8c444be97d17b19ea4316,0,0,0,0,0,0,0,0,0,0
1,0x005b9f4516f8e640bbe48136901738b323c53b00,0,0,0,0,0,0,0,0,0,0
2,0x0153775362c3071c1860e8dbfd53ccc82fa226f5,0,0,0,0,0,0,0,0,0,0
3,0x015f17e2851ea392351b7f259e4cfd03dc443cef,0,0,0,0,0,0,0,0,0,0
4,0x0167409e6106ec3e3f05a09fcf04606918d21ad5,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_iterated=pd.read_csv ('/content/iteratedcolumns.csv')
df_iterated.rename({'address_reviewed': 'Address'}, axis=1, inplace=True)
df_iterated.shape

(691, 11)

#### Creating a DataFrame
For those addresses that did have a transaction we explore and valuate in USD the tokens and native currency transferred. We divide this exploration in two due to the limitations of the free Coingecko API that we use to extract the prices.   

In [ ]:
address_reviewed=[]
Total_ERC20_tnxs=[]
ERC20_total_Ether_received=[]
ERC20_total_ether_sent=[]
ERC20_uniq_sent_addr=[]#Number of ERC20 token transactions sent to Unique account addresses
ERC20_uniq_rec_addr=[] #Number of ERC20 token transactions received from Unique addresses
ERC20_uniq_rec_contract_addr=[]#Number of ERC20 token transactions received from Unique contract addresses
ERC20_min_val_rec=[] #Minimum value in Ether received from ERC20 token transactions for account
ERC20_avg_val_rec=[]
ERC20_uniq_sent_token_name=[]

for add in new_list_to_iterate:
  address_reviewed.append(add)
  time.sleep (1)
  url= 'https://api.etherscan.io/api?module=account&action=tokentx&address='+add+'&startblock=0&endblock=17059560&sort=asc&apikey='+api
  response= requests.get (url)
  data=response.json()
  tokens_unique_rec=[]
  rec_add=[]
  sent_add=[]
  contract_add=[]
  ether_received=[]
  ether_sent=[]
  Total_ERC20_tnxs.append (len (data['result']))
  for res in range (len (data['result'])):
    token= (data['result'][res]['tokenName']).lower()
    token=token.replace(" ", "-")
    decimal=int (data['result'][res]['tokenDecimal'])
    ts=int(data['result'][res]['timeStamp'])
    date= datetime.utcfromtimestamp(ts).strftime('%d-%m-%Y')
    coingeckourl='https://api.coingecko.com/api/v3/coins/'+token+'/history?date='+date+'&localization=false'
    try:
      eth_response = requests.get(coingeckourl).json()['market_data']['current_price']['eth']
    except KeyError:
      eth_response=0
      if add ==data['result'][res]['to']:
        tokens_unique_rec.append (data['result'][res]['tokenSymbol'])
        rec_add.append  (data['result'][res]['from'])
        contract_add.append  (data['result'][res]['contractAddress'])
        value_received=(int(data['result'][res]['value'])*10**-18)*eth_response
        ether_received.append (value_received)
      elif add ==data['result'][res]['from']:
        sent_add.append (data['result'][res]['from'])
        ether_sent.append (eth_response)
  ERC20_uniq_sent_token_name.append (len(set(tokens_unique_rec)))
  ERC20_uniq_rec_addr.append(len(set(rec_add)))
  ERC20_uniq_rec_contract_addr.append(len(set(contract_add)))
  ERC20_total_Ether_received.append (sum (ether_received))
  ERC20_min_val_rec.append (min (ether_received))
  ERC20_avg_val_rec.append (mean (ether_received))
  ERC20_uniq_sent_addr.append(len(set(sent_add)))
  ERC20_total_ether_sent.append (sum (ether_sent))

In [ ]:
df_iterated_2 = pd.DataFrame(list(zip(address_reviewed, Total_ERC20_tnxs, ERC20_total_Ether_received, ERC20_total_ether_sent, ERC20_uniq_sent_addr, ERC20_uniq_rec_addr,
                           ERC20_uniq_rec_contract_addr, ERC20_min_val_rec, ERC20_avg_val_rec, ERC20_uniq_sent_token_name)),
               columns =['address_reviewed', 'Total_ERC20_tnxs', 'ERC20_total_Ether_received', 'ERC20_total_ether_sent', 'ERC20_uniq_sent_addr', 'ERC20_uniq_rec_addr',
                           'ERC20_uniq_rec_contract_addr', 'ERC20_min_val_rec', 'ERC20_avg_val_rec', 'ERC20_uniq_sent_token_name'])

df_iterated_2.head()
#df_iterated_2.to_csv ('iteratedcolumns.csv', index=False)

,address_reviewed,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_rec_contract_addr,ERC20_min_val_rec,ERC20_avg_val_rec,ERC20_uniq_sent_token_name
0,0x0400f72bca103add12c6793d6194d041983e150a,1,0.0,0,0,1,1,0.0,0.0,1
1,0x0642e5db00442400868546d5f8e21581dfe4f072,1,0.0,0,0,1,1,0.0,0.0,1
2,0x075dc757633700b257e090d62b62aac5997a99d4,1,0.0,0,0,1,1,0.0,0.0,1
3,0x09faf25e57abd0a401bb5a2341d7f926c389f8d1,1,0.0,0,0,1,1,0.0,0.0,1
4,0x0ab4fc631d194bb135dc7a3f99580cf7df3eeae0,1,0.0,0,0,1,1,0.0,0.0,1


In [ ]:
#df_iterated_2=pd.read_csv ('/content/df_iterated_2.csv')
df_iterated_2.rename({'address_reviewed': 'Address'}, axis=1, inplace=True)
df_iterated_2.shape

(134, 10)

#### Combining DataFrames
The code in this cell will combine the `df_iterated` and `df_iterated_2` DataFrames. List and store it in the `address_reviewed_list` variable.

In [ ]:
processed_df= df_iterated.append(df_iterated_2, ignore_index=True)
processed_df.shape

<ipython-input-5-7b94ceb1820c>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_df= df_iterated.append(df_iterated_2, ignore_index=True)


(825, 11)

In [ ]:
address_reviewed_list=processed_df.Address.to_list()
len (address_reviewed_list)

825

#### Filtering the Dataframe
The code in this cell will filter the `df` dataframe to remove any rows that have an address that is in the `address_reviewed_list` list. 

In [ ]:
df_sin = df[~df['Address'].isin(address_reviewed_list)]
df_sin.shape

(3847, 32)

#### Filtering the empty dataframe 
Filter the `df_onlyempty` dataframe further to only include rows that have an address that is in the `address_reviewed_list` list. Drop unnecessary columns based on decisitions taken on the EDA. 

In [ ]:
df_onlyempty = df[df['Address'].isin(address_reviewed_list)]
df_onlyempty.shape

(824, 32)

In [ ]:
df_onlyempty.drop (columns=['Total_ERC20_tnxs', 'ERC20_total_Ether_received', 'ERC20_total_ether_sent', 'ERC20_total_Ether_sent_contract', 'ERC20_uniq_sent_addr', 'ERC20_uniq_rec_addr',
                           'ERC20_uniq_rec_contract_addr', 'ERC20_min_val_rec', 'ERC20_avg_val_rec', 'ERC20_uniq_sent_token_name'], inplace= True)

<ipython-input-10-fb9e86eeb9a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_onlyempty.drop (columns=['Total_ERC20_tnxs', 'ERC20_total_Ether_received', 'ERC20_total_ether_sent', 'ERC20_total_Ether_sent_contract', 'ERC20_uniq_sent_addr', 'ERC20_uniq_rec_addr',


In [ ]:
df_onlyempty.shape

(824, 22)

#### Merge Dataframes
Merge the two dataframes and assign it to `df`.

In [ ]:
df_onlyempty= pd.merge(df_onlyempty, processed_df, on='Address')
df_onlyempty.shape

(824, 32)

In [ ]:
df_sin= df_sin.append(df_onlyempty, ignore_index=True)
df_sin.head()

<ipython-input-14-ebbe01ac1e6e>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sin= df_sin.append(df_onlyempty, ignore_index=True)


,Index,Address,FLAG,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,...,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_rec_contract_addr,ERC20_min_val_rec,ERC20_avg_val_rec,ERC20_uniq_sent_token_name,valid_add
0,2,0x002bf459dc58584d58886169ea0e80f3ca95ffaf,1,3976.50,834.77,9622.53,2,2,0,1,...,1.3370,0.0000,0.0,0.0,1.0,1.0,1.337,1.337000,0.0,0x002Bf459dC58584D58886169EA0E80f3Ca95FFAF
1,3,0x002f0c8119c16d310342d869ca8bf6ace34d9c39,1,112.90,31.87,321.42,2,3,0,3,...,1.3370,0.0000,0.0,0.0,1.0,1.0,1.337,1.337000,0.0,0x002f0C8119C16D310342D869Ca8bF6AcE34D9c39
2,4,0x0059b14e35dab1b4eee1e2926c7a5660da66f747,1,2300.37,65.10,73091.00,29,98,0,89,...,142677.3829,120354.7684,0.0,6.0,55.0,37.0,0.000,1954.484697,22.0,0x0059b14e35daB1b4EEe1e2926C7A5660dA66F747
3,6,0x00650a784ee109797ba8c05f3496bdbf2b2b7a1c,1,59129.36,0.00,118258.72,2,1,0,1,...,14.0000,0.0000,0.0,0.0,1.0,1.0,14.000,14.000000,0.0,0x00650a784EE109797BA8C05F3496bDbF2b2B7a1C
4,7,0x0084515449b037205a33d6d3940a5684126aa4b5,1,1819.90,638.12,8744.77,2,8,0,7,...,13.3700,0.0000,0.0,0.0,1.0,1.0,13.370,13.370000,0.0,0x0084515449B037205A33d6d3940a5684126Aa4b5


In [ ]:
df_sin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4681 entries, 0 to 4680
Data columns (total 50 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Index                                                  4681 non-null   int64  
 1   Address                                                4681 non-null   object 
 2   FLAG                                                   4681 non-null   int64  
 3   Avg_min_between_sent_tnx                               4681 non-null   float64
 4   Avg_min_between_received_tnx                           4681 non-null   float64
 5   Time_Diff_between_first_and_last_(Mins)                4681 non-null   float64
 6   Sent_tnx                                               4681 non-null   int64  
 7   Received_Tnx                                           4681 non-null   int64  
 8   Number_of_Created_Contracts                     

In [ ]:
df=df_sin

#### Drop Columns
Drop the columns `Index`, `Address`, and `valid_add` from the dataframe. 

In [ ]:
df.drop (columns=['Index', 'Address', 'valid_add'], inplace= True)

#### Fill Columns
Fill any missing values with the median of the column.

In [ ]:
df = df.fillna(df.median())

#### Export the Dataframe
Export the dataframe `df` to a csv file named `final.csv`. 

In [ ]:
df.to_csv ('final.csv', index=False)